# Этап 1. Получение данных

Изучим данные, предоставленные для проекта.

In [2]:
# импорт библиотеки pandas
import pandas as pd

Прочитаем файл games.csv и сохраним его в переменной df. 

In [3]:
# чтение файла с данными с сохранением в df
df = pd.read_csv('/datasets/games.csv')

Получение первых 10 строк таблицы.

In [4]:
# получение первых 10 строк таблицы.
df.head(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


Общая информация о данных таблицы df.

In [5]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
Name               16713 non-null object
Platform           16715 non-null object
Year_of_Release    16446 non-null float64
Genre              16713 non-null object
NA_sales           16715 non-null float64
EU_sales           16715 non-null float64
JP_sales           16715 non-null float64
Other_sales        16715 non-null float64
Critic_Score       8137 non-null float64
User_Score         10014 non-null object
Rating             9949 non-null object
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 11 столбцов. Присутствуют два типа данных: non-null object (5 столбцов) и non-null float64 (6 столбцов).

Подробно разберём, какие в df столбцы и какую информацию они содержат:
* Name — название игры
* Platform — платформа
* Year_of_Release — год выпуска
* Genre — жанр игры
* NA_sales — продажи в Северной Америке (миллионы долларов)
* EU_sales — продажи в Европе (миллионы долларов)
* JP_sales — продажи в Японии (миллионы долларов)
* Other_sales — продажи в других странах (миллионы долларов)
* Critic_Score — оценка критиков (от 0 до 100)
* User_Score — оценка пользователей (от 0 до 10)
* Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.


**Выводы**

Каждая строка таблицы содержит исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation), год выпуска и присвоенный возрастной рейтинг.

Две проблемы, которые нужно решать на следующем этапе: пропуски и некачественные названия столбцов. 

Для проверки рабочих гипотез особенно ценны столбцы Genre, Platform, Year_of_Release и разбивка продаж по регионам. По ним мы сможем отследить, как менялись предпочтения пользователей со времением, составить портрет пользователя каждого региона. Столбцы Critic_Score, User_Score, Rating помогут проверить гипотезы о влиянии рейтинга игр на продажи в зависимости от жанра и платформы выхода игр.

# Этап 2. Предобработка данных

Заменим названия столбцов, преобразуем данные в нужные типы, обработаем пропуски (с заполнением, где возможно), проверим данные на наличие дубликатов, а также посчитаем суммарные продажи во всех регионах и запишем их в отдельный столбец.

Получаем перечень названий столбцов.

In [9]:
# перечень названий столбцов таблицы df
df.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'NA_sales', 'EU_sales',
       'JP_sales', 'Other_sales', 'Critic_Score', 'User_Score', 'Rating'],
      dtype='object')

Названия столбцов с заглавной буквы, что не совсем удобно для доступа к данным. Переименуем столбцы для удобства дальнейшей работы. Проверим результат.

In [10]:
# переименование столбцов
df = df.rename(columns=lambda x: x.lower())

In [11]:
# проверка результатов - перечень названий столбцов
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [14]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

0

Дубликатов не обнаружено. Займемся пропущенными значениями.

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [15]:
# суммарное количество пропусков в таблице df
df.isnull().sum()

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         6701
rating             6766
dtype: int64